# Домашнее задание № 3. Исправление опечаток

In [3]:
from collections import Counter

In [4]:
import textdistance
import re

In [56]:
from string import punctuation

In [5]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [7]:
corpus = open('wiki_data.txt', encoding='utf8').read()

In [8]:
# создаем словарь
vocab = Counter(re.findall('\w+', corpus.lower()))


In [12]:
word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}




In [147]:
vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [117]:
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N


def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        met = metric.normalized_similarity(text, word) 
        similarities[word] = met
    return similarities.most_common(topn)

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

def get_closest_hybrid_match(text, X=X, vec=vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    closest=Counter(dict(closest)) #не совсем понимаю, почему, но, хотя return у метрик матч каунтер, в этой функции он становится списком. делала проверку через принт типа. буду признательна, если объясните:)
    correction, max_sim = closest.most_common(1)[0]
    for i in closest:
        if probs[correction]<probs[i] and closest[i]==max_sim:
            correction=i
    return correction



def predict_mistaken(word, vocab=vocab):
    return 0 if word in vocab else 1


## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [19]:
probs = {x:P(x) for x in vocab}

In [23]:
'word'[:0]

''

In [26]:


def deletion(word, dct):
    for l in range(len(word)):
        rmv = word[:l]+word[l+1:]
        if rmv not in dct.keys() or probs[dct[rmv]]<probs[word]: #добавляем проверку на частотность уже на этапе создания словаря удалений
            dct[rmv]=word
    


In [29]:
deletions = {}
for i in vocab:
    deletion(i, deletions)


In [121]:
def symspell(word, deletions=deletions):
    dels = dict()
    deletion(word, dels)
    try:
        return Counter({deletions[i]:probs[deletions[i]] for i in dels if i in deletions}).most_common(1)[0][0]
    except:
       # print(word)
        return word




In [90]:
Counter({'a':0,'b':0}).most_common()

[('a', 0), ('b', 0)]

In [120]:
symspell('симпатичнейшое')

симпатичнейшое


'симпатичнейшое'

In [53]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [57]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [67]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], symspell(pair[1]))
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1
    
    if not i % 100:
        print(i)

симпатичнейшое
superheadz
0
услышанная
полчатся
оччччень
нащщот
отсуствие
аварийно-спасательных
дорожно-транспортных
почте.ру
предлагю
сегодяшнее
люминала
поффтыкав
компютерная
выговаривает
капулетти
учавствовать
притащиться
оплеух
отвественность
пивоварах
павзрослому
прыщами
подсаживаеться
загруживаюсь
какието
хихикать
ощушаю
монголойдом
что-то
коментариев
что-то
по-моему
плоскостопых
обълись
ввеселились
иностранцев-гостей
зубодробительня
каааак
хочеться
супер-маркете
серебрянные
дороговато
мечталось
ооочень
доделала
легитимизирует
мущщину
оооочень
симаатическая
nettrader
припораты
варикоза
как-то
приветствутется
navision
axapta
расжился
однажэды
обезумевшая
раскидала
пользоваццо
сочуствующие
бредовую
покатавшись
креатифф
стричься
какой-то
основыные
что-то
получаеться
разрушаеться
кроординирует
расказчик
открыывю
написанно
высыпвние
перессказывать
философско-антропологической
подоплеке
каком-никаком
собстно
священослужители
огроменных
неососознанно
кое-что
попрежнему
угадаете
когдато


In [68]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8560280140070035
0.19953416149068323
0.04685884920179166


## *3. Настройка гиперпараметров. (2 балла)

У метода из первого заданий много гиперпараметров (те которые нужно подбирать самостоятельно). Это параметры векторайзера, topn, метрика редактирования, токенизация. Поэкспериментируйте с ними. 

Проведите как минимум 10 экспериментов с разными параметрами. Для каждого эксперимента укажите мотивацию (например, "слишком маленький topn в get_closest_match_vec приводит к тому, что некоторые хорошие варианты не доходят до get_closest_match_with_metric, попробуем его увеличить")

Старайтесь получить улучшение, но если улучшить не получится, то это не страшно. Главное, чтобы эксперименты были осмысленными, а не рандомными. 

In [139]:
def test_hybrid(topn=3, vec=vec, X=X, metric=textdistance.damerau_levenshtein):
    mistakes = []
    total_mistaken = 0
    mistaken_fixed = 0

    total_correct = 0
    correct_broken = 0

    total = 0
    correct = 0

    cashed = {}
    for i in range(len(true)):
        word_pairs = align_words(true[i], bad[i])
        for pair in word_pairs:
            if predict_mistaken(pair[1], vocab):
                pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], topn=topn, metric=metric,vec=vec,X=X))
                cashed[pair[1]] = pred
            else:
                pred = pair[1]
            
                
            if pred == pair[0]:
                correct += 1
            else:
                mistakes.append((pair[0], pair[1], pred))
            total += 1
                
            if pair[0] == pair[1]:
                total_correct += 1
                if pair[0] != pred:
                    correct_broken += 1
            else:
                total_mistaken += 1
                if pair[0] == pred:
                    mistaken_fixed += 1
        
        if not i % 100:
            print(i)
    print(correct/total)
    print(mistaken_fixed/total_mistaken)
    print(correct_broken/total_correct)



Для начала соберем результаты работы первой функции (для удобства обернула тестирование в функцию)

In [130]:
test_hybrid()

0
100
200
300
400
500
600
700
800
900
0.8477238619309655
0.42701863354037267
0.09004249454461927


##### 1 эксперимент
Как и было предложено сенсеем, в качестве первого эксперимента увеличим топ-н. Немного повысилась полнота!

In [133]:
test_hybrid(topn=5) 

0
100
200
300
400
500
600
700
800
900
0.8494247123561781
0.44021739130434784
0.09004249454461927


##### Второй эксперимент.
Можно исходить из предположения, что определенные сочетания букв человек печатает чаще, следовательно - опечатки в них случаются реже! Так что для поиска было бы удобно использовать n-граммы. Триграммы как самая большая величина здесь вполне подойдут (поскольку суффиксы, приставки etc обычно обладают длиной в два-три символа). А проверка на частотность нужна для, например, некириллических символов, которые вполне логично могли утечь к нам из википедии.

И ура! Мы видим в метриках более-менее ощутимые улучшения!

In [143]:
vec_ngram = CountVectorizer(analyzer='char', ngram_range=(1,3),min_df=5, max_df=0.3)
X_ngram = vec_ngram.fit_transform(vocab)

In [146]:
test_hybrid(vec=vec_ngram,X=X_ngram) 

0
100
200
300
400
500
600
700
800
900
0.8547273636818409
0.4813664596273292
0.09004249454461927


##### Третий эксперимент
Просто сменить метрику на более простую (рассчитывается только по совпадению символов с одинаковыми индексами). Увенчался огромной неудачей, и даже скорость не увеличилась! 

In [138]:
test_hybrid(metric=textdistance.hamming)

0
100
200
300
400
500
600
700
800
900
0.8257128564282141
0.2562111801242236
0.09004249454461927


##### Четвертый эксперимент
Если верить гитхабу, jaro чуть лучше справляется с короткими словами, чем левенштейн, и можно опробовать его.

И, как мы видим, эксперимент оказался весьма удачным!

In [149]:
test_hybrid(metric=textdistance.jaro_winkler)

0
100
200
300
400
500
600
700
800
900
0.8431215607803902
0.391304347826087
0.09004249454461927


##### 5 кспрмнт
Стало интересно посмотреть на метрику, которая неожиданно пригодилась для матчинга арабских слов - mlipns. Кажется, для русского языка такой метод тоже может быть хорошо применим

Но кажется зря: результаты лишь ухудшились, хоть и не так сильно, как в случае с хаммингом.

In [150]:
test_hybrid(metric=textdistance.mlipns)

0
100
200
300
400
500
600
700
800
900
0.8296148074037019
0.2864906832298137
0.09004249454461927


##### Шестой эксперимент

Начинаем комбинировать удачные находки! Улучшенный векторайзер в сочетании со слегка увеличенным топ-н снова дали прирост результата!

In [151]:
test_hybrid(X=X_ngram,vec=vec_ngram, topn=5)

0
100
200
300
400
500
600
700
800
900
0.856328164082041
0.4937888198757764
0.09004249454461927


##### Седьмой эксперимент
Поскольку в прошлом было достигнуто улучшение, стоит попробовать собрать уже все удачные находки вместе. 

И - неудача - хоть и нефатальная. Лучше изначального результата, но хуже простого сочетания изначально используемой метрики, увеличения топ-н и нграмм.

In [152]:
test_hybrid(X=X_ngram,vec=vec_ngram, topn=5, metric=textdistance.jaro_winkler)

0
100
200
300
400
500
600
700
800
900
0.8508254127063531
0.45108695652173914
0.09004249454461927


##### Восьмой эксперимент

Хотя мы, кажется, нашли оптимальный вариант с ощутимым приростом, можно протестировать jaro-winklera на чистом векторайзере, но с увеличенным топ-н.

In [153]:
test_hybrid(topn=5, metric=textdistance.jaro_winkler)

0
100
200
300
400
500
600
700
800
900
0.8444222111055528
0.40139751552795033
0.09004249454461927


##### Девятый и десятый эксперименты

Даем фору аутсайдерам: запустим млпинс и хамминга с векторайзером и расширенным окном

Но, увы, чуда не произошло!

In [155]:
test_hybrid(X=X_ngram,vec=vec_ngram, topn=5, metric=textdistance.mlipns)

0
100
200
300
400
500
600
700
800
900
0.8193096548274137
0.20652173913043478
0.09004249454461927


In [156]:
test_hybrid(X=X_ngram,vec=vec_ngram, topn=5, metric=textdistance.hamming)

0
100
200
300
400
500
600
700
800
900
0.8279139569784892
0.2732919254658385
0.09004249454461927
